<a href="https://colab.research.google.com/github/SherbyRobotics/pyro/blob/colab/examples/notebooks/cartpole_trajectoryoptimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Optimal Control Overview

This page shows and exemple of planning an optimal trajectory for a cart-pole swing-up with the direct-collocation method.

**Importing Librairies**

This page uses the toolbox *pyro*.

In [ ]:
!git clone -b dev-alex https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')

In [ ]:
import numpy as np
from IPython import display
!apt install ffmpeg

# Defining the dynamics

Here we load a already defined class from the library including the dynamic equations of the cart-pole, which is a function of the form:

$\dot{x} = f(x,u)$

The class also inlcude other variables defining the control problem that we can set:
*   The max/min torque available;
*   The initial state;
*   The goal state;


 

In [ ]:
##############################################################################
# Dynamics
##############################################################################

from pyro.dynamic  import cartpole

# Define the dynamical system to control
sys  = cartpole.UnderActuatedRotatingCartPole()

sys.u_ub[0] = +50 # Max torque
sys.u_lb[0] = -50 # Min torque

sys.x0    = np.array([0,-3.14,0,0]) # starting states
sys.xbar  = np.array([0,0,0,0])    # goal states



The dynamical equations $\dot{x} = f(x,u)$ can be represented graphically by a vector field shown here:

In [ ]:
sys.plot_phase_plane(0,2) # Graphical illustration of the dynamic behavior in the phase plane
sys.plot_phase_plane(1,3)

Since the system has 4 states, the dynamics is actually a 4d vector field. Here we ploted 2 sub-planes of this higher dimension space.

#Defining the cost function

Most optimal control algorithm are based on optimising a "cost-function" which is a mathematical representation of the performance

Here we will use a standard quadratic cost function or the type:

$J = \int  ( x^T Q x + u^T R u ) dt$

In [ ]:
##############################################################################
# Cost Function
##############################################################################

from pyro.analysis import costfunction

cf = costfunction.QuadraticCostFunction.from_sys( sys ) 

cf.INF  = 10000     # The value iteration algo needs this parameter

cf.Q[0,0] = 1
cf.Q[1,1] = 1
cf.Q[2,2] = 100
cf.Q[3,3] = 1
cf.R[0,0] = 1

sys.cost_function = cf

Here we see the final matrix used to define the cost function:

In [ ]:
print('Q=\n',cf.Q)
print('R=\n',cf.R)

#Trajectory Optimisation

Another approach is instead of trying to find the optimal controller directly, it is much faster to find an optimal trajectory from just an initial state. One approach is to formulate the problem as an constrained optimisation problem of the form:

$min \int_{t_0}^{t_f}  ( x^T Q x + u^T R u ) dt  $

subject to $\dot{x} = f(x,u)$

In [ ]:
##############################################################################
# Direct Collocation Trajectory Optimisation
##############################################################################

from pyro.planning import trajectoryoptimisation

planner = trajectoryoptimisation.DirectCollocationTrajectoryOptimisation( sys ,
                                                                          dt = 0.1,
                                                                          grid = 30)

planner.x_start = sys.x0 
planner.x_goal  = sys.xbar

planner.compute_optimal_trajectory()


## Fail!
The optimiser failled to converge. Optimising non-linear fonction under constraints is tricky. One way to improve our chance of convergence is to guide the solver with an initial trajectory guess.

# Coarse feasible trajectory with RRT

Here we will use another type of algorithme to find a rough "feasible solution" in order to give an initial guess to our trajectory optimiser.

In [ ]:
#######################################
# Coarse planning with RRT
#######################################

from pyro.planning import randomtree

rrt = randomtree.RRT( sys , sys.x0 )

# Discrete actions
rrt.u_options = [
        sys.u_lb,
        sys.u_ub,
        np.array([ 0])
        ]

# RRT search parameters
rrt.goal_radius          = 1.5
rrt.max_nodes            = 10000
rrt.max_solution_time    = 3.0
rrt.dt                   = 0.05
rrt.max_distance_compute = 1000
rrt.dyna_plot            = False

rrt.find_path_to_goal( sys.xbar )

rrt.plot_tree()
rrt.plot_open_loop_solution()




So here we have a rough solution, but as we can see on the solution plot, it is very rought and coarse. Hence, we could not really expect to use it directly. But for an initial guess it can still be very usefull.

**Trajectory Optimisation with an initial guess solution**

Here we will again try the direct collocation method but we will give an initial guess to the solver.

In [ ]:
planner2 = trajectoryoptimisation.DirectCollocationTrajectoryOptimisation( sys ,
                                                                          dt = 0.1,
                                                                          grid = 30)
planner2.x_start = sys.x0 
planner2.x_goal  = sys.xbar

planner2.set_initial_trajectory_guest( rrt.trajectory )

planner2.compute_optimal_trajectory()

In [ ]:
planner2.show_solution()

In [ ]:
video = planner2.animate_solution_to_html( is_3d = True )
html  = display.HTML(video)
display.display(html)